In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow import keras 
from PIL import Image
import numpy as np 
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical

In [2]:

image_directory='datasets/'

no_tumor_images=os.listdir(image_directory+ 'no/')
yes_tumor_images=os.listdir(image_directory+ 'yes/')
dataset=[]
label=[]

INPUT_SIZE=64 

In [3]:

for i , image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'no/'+image_name)
        image=Image.fromarray(image,'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

for i , image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'yes/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset=np.array(dataset)
label=np.array(label)


x_train, x_test, y_train, y_test=train_test_split(dataset, label, test_size=0.2, random_state=0)


In [4]:

# Reshape = (n, image_width, image_height, n_channel)

# print(x_train.shape)
# print(y_train.shape)

# print(x_test.shape)
# print(y_test.shape)

x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

y_train=to_categorical(y_train , num_classes=2)
y_test=to_categorical(y_test , num_classes=2)

In [5]:
# Model Building
# 64,64,3

model=Sequential()

model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))



In [6]:

# Binary CrossEntropy= 1, sigmoid
# Categorical Cross Entryopy= 2 , softmax

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


model.fit(x_train, y_train, 
batch_size=16, 
verbose=1, epochs=10, 
validation_data=(x_test, y_test),
shuffle=False)


model.save('BrainTumormodel.h5')


Epoch 1/10


9/9 [==============================] - 3s 98ms/step - loss: 0.7013 - accuracy: 0.4485 - val_loss: 0.6689 - val_accuracy: 0.6857
Epoch 2/10
9/9 [==============================] - 0s 49ms/step - loss: 0.6668 - accuracy: 0.6544 - val_loss: 0.6296 - val_accuracy: 0.7143
Epoch 3/10
9/9 [==============================] - 0s 39ms/step - loss: 0.6246 - accuracy: 0.6765 - val_loss: 0.5702 - val_accuracy: 0.7429
Epoch 4/10
9/9 [==============================] - 0s 43ms/step - loss: 0.6104 - accuracy: 0.6544 - val_loss: 0.5122 - val_accuracy: 0.8000
Epoch 5/10
9/9 [==============================] - 0s 43ms/step - loss: 0.5633 - accuracy: 0.7279 - val_loss: 0.4865 - val_accuracy: 0.8000
Epoch 6/10
9/9 [==============================] - 0s 40ms/step - loss: 0.5208 - accuracy: 0.7132 - val_loss: 0.4856 - val_accuracy: 0.8000
Epoch 7/10
9/9 [==============================] - 0s 47ms/step - loss: 0.5191 - accuracy: 0.7279 - val_loss: 0.4714 - val_accuracy: 0.8286
Epoch 8/10
9/9 [========

c:\Users\rasto\Desktop\brain-tumor-detection\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
from keras.models import load_model

In [8]:
model=load_model('BrainTumormodel.h5')

image=cv2.imread('C:\\Users\\rasto\\Desktop\\brain-tumor-detection\\test-data\\pred5.jpg')

img=Image.fromarray(image)

img=img.resize((64,64))

img=np.array(img)

input_img=np.expand_dims(img, axis=0)

result=model.predict(input_img)
print(result)

1/1 [==============================] - 0s 148ms/step
[[0. 1.]]


In [9]:
from tensorflow.keras.preprocessing import image
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename

In [10]:
app = Flask(__name__)


model =load_model('BrainTumormodel.h5')
print('Model loaded. Check http://127.0.0.1:5000/')


def get_className(classNo):
	if classNo==0:
		return "No Brain Tumor"
	elif classNo==1:
		return "Yes Brain Tumor"


def getResult(img):
    image=cv2.imread(img)
    image = Image.fromarray(image, 'RGB')
    image = image.resize((64, 64))
    image=np.array(image)
    input_img = np.expand_dims(image, axis=0)
    result=model.predict(input_img)
    return result


@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        f = request.files['file']

        basepath = os.path.dirname('C:\\Users\\rasto\\Desktop\\brain-tumor-detection\\test-data')
        file_path = os.path.join(
            basepath, 'test-data', secure_filename(f.filename))
        f.save(file_path)
        value=getResult(file_path)
        if value[0][1]==1:
            value[0][0]=1
        else:
             value[0][1]=0
            
        
        res = value.all()
        result=get_className(res)
        return result
    return None


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

Model loaded. Check http://127.0.0.1:5000/
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Jan/2024 11:01:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2024 11:01:05] "GET /static/js/bootstrap.min.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Jan/2024 11:01:05] "GET /static/js/newjs.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Jan/2024 11:01:05] "GET /static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Jan/2024 11:01:05] "GET /static/css/test.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Jan/2024 11:01:05] "GET /static/js/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Jan/2024 11:01:05] "GET / HTTP/1.1" 200 -
